In [1]:
"""import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1""""

In [2]:
"""from google.colab import drive
drive.mount('/content/drive')"""

Mounted at /content/drive


In [ ]:
"""!pip install datasets tokenizers transformers"""

In [9]:
import torch
import argparse
import numpy as np
from datasets import Dataset
from scipy.special import expit
from transformers import (
    DataCollatorWithPadding, TrainingArguments,
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, AutoConfig
)

import gc
import random
import ast
import copy
import gc
import itertools
import joblib
import json
import math
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import pickle
import random
import re
import scipy as sp
import string
import sys
import time
import warnings


from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

# ======= OPTIONS =========
"""device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
warnings.filterwarnings("ignore")
!mkdir output"""

In [91]:
import polars as pl
from sklearn.model_selection import train_test_split

df_pl = pl.read_parquet("/home/rijkaa/leraa/train_essays.parquet")#-----------------------------------------------------------------------------------------------

df_pl = df_pl.unique(subset=['text'], maintain_order=True)

sample_df = df_pl.sample(fraction=1, seed=42)

train, test = train_test_split(sample_df, train_size=1, random_state=42)

In [92]:
class_0 = sample_df.filter(sample_df['generated'] == 0)
class_1 = sample_df.filter(sample_df['generated'] == 1)

count_class_0, count_class_1 = class_0.height, class_1.height

# Выбрать случайным образом примеры из класса с большим количеством
undersampled_class_1 = class_1.sample(fraction=count_class_0 / count_class_1, shuffle=True)

# Объединить примеры из обоих классов в новый DataFrame
undersampled_df = class_0.extend(undersampled_class_1)

In [94]:
train = undersampled_df
train

id,prompt_id,text,generated
str,i64,str,i64
"""e_2tl5ylwy""",0,"""H ello fellow…",0
"""e_mgv97k7z""",0,"""Limiting car u…",0
"""e_1m6o7oy4""",0,"""Cars have been…",0
"""e_79v25z8a""",0,"""Limiting car u…",0
"""e_23kxh3s4""",0,"""Cars are usual…",0
…,…,…,…
"""e_drngerx2""",7,"""When older stu…",1
"""e_kl5fdi9u""",10,"""I think it wou…",1
"""e_fw445ie6""",103,"""“How did you g…",1


In [95]:
train_df = pl.DataFrame({"text": train['text'], "label": train['generated']})
train_df

text,label
str,i64
"""H ello fellow…",0
"""Limiting car u…",0
"""Cars have been…",0
"""Limiting car u…",0
"""Cars are usual…",0
…,…
"""When older stu…",1
"""I think it wou…",1
"""“How did you g…",1


In [108]:
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AutoTokenizer

config = AutoConfig.from_pretrained("microsoft/deberta-v3-xsmall", num_labels=2)
model = AutoModelForSequenceClassification.from_config(config=config)

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall", use_fast=True)

/home/rijkaa/miniconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [109]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=128, padding=True, truncation=True)

In [110]:
train, valid = train_test_split(train_df, train_size=0.8, shuffle=True, random_state=42)
train

text,label
str,i64
"""Once upon a ti…",1
"""VENUS Venus i…",0
"""I am against t…",1
"""Dear Principal…",0
"""The virtual ex…",0
…,…
"""The developmen…",1
"""I want to chan…",0
"""Summer project…",1


In [111]:
from datasets import Dataset
ds_train = Dataset.from_pandas(train.to_pandas())
ds_valid = Dataset.from_pandas(valid.to_pandas())

ds_train_enc = ds_train.map(preprocess_function, batched=True)

ds_valid_enc = ds_valid.map(preprocess_function, batched=True)


Map:   0%|          | 0/57956 [00:00<?, ? examples/s]

Map:   0%|          | 0/14490 [00:00<?, ? examples/s]

In [112]:
metric_name = "roc_auc"
model_name = "deberta-xsmall"
train_batch_size = 4
eval_batch_size = 32
grad_acc = 4

In [113]:
num_steps = len(train) // (train_batch_size * grad_acc)
num_steps

3622

In [118]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = num_steps // 3,
    save_steps = num_steps // 3,
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
)

In [119]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    auc = roc_auc_score(labels, probs[:,1], multi_class='ovr')
    return {"roc_auc": auc}

In [120]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_train_enc,
    eval_dataset=ds_valid_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/rijkaa/miniconda3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [121]:
trainer.train()

Step,Training Loss,Validation Loss,Roc Auc
1207,0.454200,0.503043,0.831882
2414,0.448800,0.546434,0.833326
3621,0.453300,0.490293,0.847735
4828,0.386100,0.481796,0.851831
6035,0.398400,0.518759,0.854084
7242,0.406700,0.472342,0.863946
8449,0.338200,0.481390,0.856681
9656,0.362100,0.474139,0.865684
10863,0.353400,0.493872,0.868866
12070,0.314600,0.504193,0.869951


TrainOutput(global_step=18110, training_loss=0.36279603188239296, metrics={'train_runtime': 8288.2927, 'train_samples_per_second': 34.963, 'train_steps_per_second': 2.185, 'total_flos': 4771991476715520.0, 'train_loss': 0.36279603188239296, 'epoch': 5.0})